In [10]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from pinecone import Pinecone, ServerlessSpec

In [7]:
import os

In [8]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [9]:
pc = Pinecone(api_key=pinecone_api_key)

In [11]:
index_name = "hybrid-search-langchain-pinecone"

In [12]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='dotproduct',
        spec=ServerlessSpec(cloud="aws",region="us-east-1"),
    )

In [13]:
index = pc.Index(index_name)
index

C:\Users\004NQ8744\OneDrive\Roadwork\langChain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [15]:
from langchain_huggingface import HuggingFaceEmbeddings

In [16]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [17]:
from pinecone_text.sparse import BM25Encoder

In [18]:
bm25_encoder = BM25Encoder().default()
bm25_encoder

In [20]:
sentences = [
    "In 2025, I worked in IBM",
    "In 2022, I worked in Assurant",
    "In 2021, I worked in CodeQuest"
]

In [21]:
bm25_encoder.fit(sentences)

100%|██████████| 3/3 [00:00<00:00, 16.60it/s]


In [22]:
bm25_encoder.dump("bm25_values.json")

In [23]:
bm25_encoder = BM25Encoder().load("bm25_values.json")

In [24]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)

In [25]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001C4B4A45A90>, index=<pinecone.db_data.index.Index object at 0x000001C486474550>)

In [26]:
retriever.add_texts(
    [
    "In 2025, I worked in IBM",
    "In 2022, I worked in Assurant",
    "In 2021, I worked in CodeQuest"
    ]
)

100%|██████████| 1/1 [00:02<00:00,  2.41s/it]


In [27]:
retriever.invoke("What jon did I join first?")

[Document(metadata={'score': 0.0605011}, page_content='In 2022, I worked in Assurant'),
 Document(metadata={'score': 0.0549702644}, page_content='In 2021, I worked in CodeQuest'),
 Document(metadata={'score': 0.0376462936}, page_content='In 2025, I worked in IBM')]